<style>
@import url(https://www.numfys.net/static/css/nbstyle.css);
</style>
<a href="https://www.numfys.net"><img class="logo" /></a>

# Распространение одномерных волновых пакетов
## Examples - Quantum Mechanics
<section class="post-meta">
By Jonas Tjemsland and Martin Duy Tat
</section>
Last edited: January 20th 2019
___

Целью этого примера является изучение численного интегрирования уравнения Шредингера, распространения волновых пакетов, рассеяния и туннелирования через барьер. Начальные условия исследуемых волновых пакетов представлены гауссовым распределением плоских волн в области, в которой $V=0$, т.е.

\begin{equation}
\Psi(x,t)=C\exp\left[-\frac{(x-x_s)^2}{2\sigma^2}\right]
		\exp\left[i(k_0x-\omega_0 t)\right].
\end{equation}

Здесь $C$ - константа нормализации, $x_s=\langle x\rangle_{t=0}$ - начальное смещение Гаусса, $\sigma$ - ширина Гаусса (здесь ширина в половине максимума равна $2\sqrt{2\ln 2}\sigma$ и $\Delta x = \sigma$), $k_0\equiv p_0/\hbar$ - волновое число и $\omega_0\equiv E/\hbar=\hbar k_0^2/2 м$ - это угловая частота волны. Можно показать, что $\Psi(x,t)$ для $t>0$ и $V=0$ на самом деле должны быть

\begin{equation}
\Psi(x,t)=\frac{C\sigma}{\sqrt{\sigma^2+it\hbar/m}} e^{i[k_0x-\omega_0t]}\exp\left\{-\frac{(x-x_s-k_0\hbar t/m)^2}{2\sigma+ 2it\hbar /m}\right\}.
\end{equation}

Выбранное представление, однако, является хорошим приближением для начальных условий, поскольку метод, используемый в этом блокноте, требует начального состояния действительной части и мнимой части в два разных момента времени, поэтому мы принимаем, что это дает небольшую ошибку.

Теперь мы импортируем необходимые библиотеки Python и задаем параметры графика:

In [ ]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import rc

# Set common figure parameters:
newparams = {'axes.labelsize': 14, 'axes.linewidth': 1, 'savefig.dpi': 300, 
             'lines.linewidth': 1.0, 'figure.figsize': (6, 2),
             'figure.subplot.wspace': 0.4,
             'ytick.labelsize': 10, 'xtick.labelsize': 10,
             'ytick.major.pad': 5, 'xtick.major.pad': 5,
             'legend.fontsize': 10, 'legend.frameon': False, 
             'legend.handlelength': 1.5, 'figure.dpi': 150}
plt.rcParams.update(newparams)

### Теория

Зависящее от времени уравнение Шедингера в одном измерении гласит,

\begin{equation}
i\hbar\frac{\partial}{\partial t}\Psi(x,t)=-\frac{\hbar^2}{2m}\frac{\partial^2}{\partial x^2}\Psi(x,t)+V(x)\Psi(x,t).
\end{equation}

Используя обозначение $\Psi(x,t)=\Psi_R(x,t)+i\Psi_I(x,t)$, мы и можем разделить уравнение Шредингера на действительную часть,

\begin{equation}
-\hbar\frac{\partial}{\partial t}\Psi_I(x,t)=-\frac{\hbar^2}{2m}\frac{\partial^2}{\partial x^2}\Psi_R(x,t)+V(x)\Psi_R(x,t),
\end{equation}

и мнимую часть,

\begin{equation}
\hbar\frac{\partial}{\partial t}\Psi_R(x,t)=-\frac{\hbar^2}{2m}\frac{\partial^2}{\partial x^2}\Psi_I(x,t)+V(x)\Psi_I(x,t).
\end{equation}

Мы видим, что приведенные выше уравнения соединяют действительную и мнимую части уравнения Шредингера.
Используя численное приближение для дифференцирования,

\begin{equation}
\frac{\partial}{\partial x}f(x)\approx\frac{f(x+\frac{\Delta x}{2})-f(x-\frac{\Delta x}{2})}{\Delta x},
\end{equation}

можно переписать воображаемую и реальную части следующим образом

\begin{equation}
\begin{split}
		\Psi_I(x,t+\frac{\Delta t}{2})=\Psi_I(x,t-\frac{\Delta t}{2})-\Delta t \left[ \frac{V(x)}{\hbar}\Psi_R(x,t)-\frac{\hbar}{2m}\frac{\Psi_R(x+\Delta x,t)-2\Psi_R(x,t)+\Psi_R(x-\Delta x,t)}{(\Delta x)^2} \right],
	\end{split}
\end{equation}

и

\begin{equation}
\begin{split}
		\Psi_R(x,t+\Delta t)=\Psi_R(x,t)-\Delta t \left[ \frac{V(x)}{\hbar}\Psi_I(x,t+\frac{\Delta t}{2})- \frac{\hbar}{2m}\frac{\Psi_I(x + \Delta x,t+\frac{\Delta t}{2})-2\Psi_I(x,t+\frac{\Delta t}{2})+\Psi_I(x-\Delta x,t+\frac{\Delta t}{2})}{(\Delta x)^2}\right].
	\end{split}
\end{equation}

Эти уравнения легко приводят к процедуре нахождения $\Psi_I(x,t+\frac{\Delta t}{2})$ когда $\Psi_I(x,t-\frac{\Delta t}{2})$ и $\Psi_R(x,t)$ известны, и точно так же для поиска $\Psi_R(x,t+\Delta t)$ когда $\Psi_R(x,t)$ и $\Psi_I(x,t-\frac{\Delta t}{2})$ известны. Эта процедура, естественно, приводит к итерационной процедуре численного распространения волнового пакета.



### Численная стабильность

Поскольку уравнение Шредингера решается численно, очевидно, что решение является только приближением истинного решения. Поэтому важно выбрать $\Delta t$ и $\Delta x$ как можно меньше, чтобы получить желаемую точность. Однако также важно учитывать взаимосвязь между $\Delta t$ и $\Delta x$. Неправильный выбор этих параметров не только снизит точность, но и ошибка может увеличиваться экспоненциально для каждой итерации.

Переставляя формулу для численного дифференцирования, использованную выше, мы получаем

\begin{equation}
\Psi(x,t+\Delta t)-\Psi(x,t)=\Delta t \frac{\partial}{\partial t}\Psi(x,t).
\end{equation}

Поскольку $\Delta t$ является лишь приближением к бесконечно малому времени, очевидно, что при использовании этого выражения будет допущена ошибка.
Относительное изменение волновой функции $\Psi(x,t)$ за интервал времени $\Delta t$ определяется

\begin{equation}
\frac{\Psi(x,t+\Delta t)-\Psi(x,t)}{\Psi(x,t)}=\Delta t\frac{\frac{-\hbar^2}{2m}\frac{\partial^2}{\partial x^2}\Psi(x,t)+V(x)\Psi(x,t)}{i\hbar\Psi(x,t)}.
\end{equation}

Наибольшая возможная пространственная кривизна для волновой функции, $\frac{\partial^2}{\partial x^2}\Psi(x,t)$, произойдет, если волновая функция изменится от одного пика к соседнему пику на расстоянии $2\Delta x$. Это подразумевает, что

\begin{equation}
\left| \frac{\partial^2}{\partial x^2}\Psi(x,t) \right| < \left|\frac{\Psi(x,t)}{(\Delta x)^2} \right| .
\end{equation}

$\Delta t$ должно быть выбрано таким образом, чтобы доля, в которой волновая функция, $\Psi(x, t)$, изменяется в течение интервала времени $\Delta t$, была намного меньше 1. Такая зависимость выводится из двух последних уравнений,
 
\begin{equation}
\Delta t \ll \frac{2m\hbar(\Delta x)^2}{\hbar^2+2m(\Delta x)^2V_{max}},
\end{equation}

где $V_{max}$ - максимальное значение потенциала в нашем домене. Мы будем использовать этот критерий для обеспечения численной стабильности.

### Численная реализация

Все величины выбираются безразмерным образом. Определенная пространственная область равна $x\in [0,L]$, где сделан выбор $L=20$. Кроме того, используются атомная система единиц, что означает $\hbar=1$ и $m=1$. Волновое число равно $k_0=20$, ширина равна $\sigma = 1$, а начальное смещение Гаусса равно $x_s=5$. Используя этот выбор, весь пакет волн включается в заданную пространственную область и легко заполняется половиной заданной ширины после $t_{max}=L/2v_g$, где $v_g$ задается

\begin{equation}
\left. v_g=\frac{\partial \omega}{\partial k}\right|_{k_0}.
\end{equation}
    
Теперь мы определяем константы, выбранные выше:

In [ ]:
L = 20.0                       # length of box
hbar = 1.0                     # hbar value, atomic units
m = 1.0                        # mass, atomic units
sigma = 1.0                    # width of the Gaussian
k0 = 20.0                      # the wave number
xs = 5.0                       # start displacement of the Gaussian
C=1/(1.77156762398*sigma)**0.5 # normalization constant
E = hbar**2*k0**2/(2*m)        # Energy of the wave packet
w = E/hbar                     # angular frequency
vg = hbar*k0/m                 # group velocity
tmax = L/(2*vg)                # time when the wave have propagated approx. half the box width

Необходимо также определить приращение времени и длины и создать вектор, содержащий все значения $x$ в заданном интервале. Обратите внимание, что критерии численной стабильности справедливы для этого выбора значений параметров.

In [ ]:
dx = 0.01                # length increment
dt = 5*10**(-5)          # time increment
n = int(L/dx)            # number of x-values (=len(x))
x = np.linspace(0, L, n) # x-values on the x-axis

Теперь пришло время вычислить $\Psi_I(x,-\frac{\Delta t}{2})$ и $\Psi_R(x,0)$. Более того, необходимо определить вектор для потенциала. Мы также хотим рассчитать распределение вероятностей, $|\Psi(x,t)|^2=\Psi_I^2+\Psi_R^2$.

Определив мнимую и действительную части волновой функции таким образом,что $\Psi(0,t)\equiv\Psi(L, t)\equiv 0$, получается тот же результат, как если бы потенциал был бесконечностью для $x\not\in(0,L)$. Другими словами, это коробка длиной $L$.

In [ ]:
PsiI = C*np.exp(-(x-xs)**2/(2*sigma**2))*np.sin(k0*x-w*(-dt/2))  #\Psi_I(x,-dt/2)
PsiR = C*np.exp(-(x-xs)**2/(2*sigma**2))*np.cos(k0*x-w*0)        #\Psi_R(x,0)
probPsi = PsiI**2 + PsiR**2  #|\Psi|^2
V = np.array([0]*n)          # Potential
PsiI[1] = PsiI[n-1] = PsiR[0] = PsiR[n-1] = 0

Можно было бы нормализовать волновую функцию, решив таким образом $C$. Он рассчитывается путем интегрирования распределения вероятностей $|\Psi(x,t)|^2=\Psi_I^2+\Psi_R^2$ над полем и выбора $C$ таким образом, чтобы $|\Psi(x,t)|^2=1$. Нормализация, скорее всего, немного изменится по мере распространения волны (и если будут выбраны другие базовые константы) из-за численных ошибок. Однако мы предполагаем, что это изменение незначительно. Ниже мы покажем, что волновая функция близка к нормализации с выбором $C$, определенным выше.

In [ ]:
f = np.sum(probPsi*dx)
print('Normalization yields total probability equal to %0.15f' % f)

### Начальное Условие

Теперь мы построим действительную и мнимую части волновой функции вместе с распределением вероятностей при $t=0$.

In [ ]:
plt.plot(x/L, PsiI, label="$\Psi_I(x,0)$", linewidth=0.5)
plt.plot(x/L, PsiR, label="$\Psi_R(x,0)$", linewidth=0.5)
plt.plot(x/L, probPsi,'m', label="$|\Psi(x,0)|^2$")
plt.axis([0, 1,-1,1])
plt.legend(prop={'size':8})
plt.xlabel('$x/L$');

### Распространение при $V=0$

Теперь мы определяем функцию `timestep()`, которая распространяет действительную и мнимую части волновой функции вперед во времени с одним временным шагом. 

In [ ]:
def timestep(PsiI, PsiR, V, dx, dt):
    """ This function takes the current PsiI and PsiR and calculates the next value
    according to the theory section. 
    
    Input:
        PsiI    # Imaginary part of wavefunction at current timestep
        PsiR    # Real part of wavefunction at current timestep
        V       # Array with value of potential in the domain
        dx      # Grid size
        dt      # Size of time step
    
    Output:
        PsiI    # Imaginary part of wavefunction at next timestep
        PsiR    # Real part of wavefunction at next timestep
    """
    
    PsiI[1:n-1] = PsiI[1:n-1] - dt*(
        PsiR[1:n-1]*V[1:n-1]/hbar - hbar*(PsiR[2:n] - 2*PsiR[1:n-1] + PsiR[0:n-2])/(2*m*dx**2))
    PsiR[1:n-1] = PsiR[1:n-1] + dt*(
        PsiI[1:n-1]*V[1:n-1]/hbar - hbar*(PsiI[2:n] - 2*PsiI[1:n-1] + PsiI[0:n-2])/(2*m*dx**2))
    return PsiI, PsiR

Если этот расчет выполняется $t_{max}/\Delta t$ раз, волновой пакет распространяется примерно на половину ширины коробки. Делая это для разных значений $\sigma$, можно получить представление о том, как распределяется волновой пакет при его распространении. Можно ожидать, что волновой пакет будет распространяться больше, если $\sigma$ мал.

In [ ]:
# Set values of sigma
sigmaval = [0.5 , 1, 1.5, 2]

for i in range(len(sigmaval)):

    PsiI = C*np.exp(-(x - xs)**2/(2*sigmaval[i]**2))*np.sin(k0*x - w*(-dt/2)) #\Psi_I(x,-dt/2)
    PsiR = C*np.exp(-(x - xs)**2/(2*sigmaval[i]**2))*np.cos(k0*x - w*0) #\Psi_I(x,0)
    PsiI[1] = PsiI[n-1] = PsiR[0] = PsiR[n-1] = 0
    
    # Plot for t=0
    plt.figure(i+1)
    probPsi = PsiR**2 + PsiI**2
    plt.plot(x/L, probPsi, linewidth=0.5, label='$t=0$')
    
    # Propagate wavefunction in time
    for j in range(int((tmax)/dt)):
        PsiI, PsiR = timestep(PsiI, PsiR, V, dx, dt)
    
    # Plot probability distribution at t = t_max
    probPsi = PsiR**2 + PsiI**2
    plt.plot(x/L, probPsi, linewidth=0.5, label='$t=t_{max}$')
    plt.legend(prop={'size':10})
    plt.axis([0, 1, -1, 1])
    plt.xlabel('$x/L$')
    plt.ylabel('$|\Psi|^2$')
    titletext = ('$\sigma$ = ' + str(sigmaval[i]))
    plt.title(titletext);

### Конечный $V$

Теперь мы вводим потенциальный барьер в нашу систему. Когда вводится потенциальный барьер $V(x)$, волновой пакет будет частично отражен и частично передан. Для начала квадратный барьер выбирается таким образом, чтобы

\begin{equation}
		V(x)=
		\begin{cases}
    			E/2,& \text{if } x\in[\frac{24}{50}L,\frac{26}{50}L],\\
			0,& \text{otherwise.}
        \end{cases}
\end{equation}

Скрипт можно легко изменить для работы с любым барьером, просто изменив $V(x)$.

In [ ]:
# Setting the potential
V = np.array([0]*n)
V0 = E/2
length = L/50
V[int(n/2-n*length/(2*L)):int(n/2+n*length/(2*L))] = V0

# Propagating wave function
PsiI = C*np.exp(-(x - xs)**2/(2*sigma**2))*np.sin(k0*x - w*(-dt/2)) #\Psi_I(x,-dt/2)
PsiR = C*np.exp(-(x - xs)**2/(2*sigma**2))*np.cos(k0*x - w*0) #\Psi_I(x,0)
PsiI[1] = PsiI[n-1] = PsiR[0] = PsiR[n-1] = 0

for j in range(int((tmax)/dt)):
    PsiI, PsiR = timestep(PsiI, PsiR, V, dx, dt)

probPsi = PsiI**2 + PsiR**2

# Plot probability distribution at t = t_max
plt.plot(x/L, probPsi,'m', label='$|\Psi(x)|^2$' )
plt.plot(x/L, V/E,'k', label='$V(x)$')
plt.axis([0, 1, -1, 1])
plt.xlabel('$x/L$')
plt.legend(prop={'size':10});

Коэффициент пропускания может быть найден численно путем интегрирования вероятностного распределения от $L/2$ до $L$, а также для коэффициента отражения путем интегрирования от $0$ до $L/2$. Сумма коэффициентов пропускания и отражения должна быть равна единице.

In [ ]:
T = sum(probPsi[int(n/2):int(n-1)]*dx)/sum(probPsi*dx)
R = sum(probPsi[0:int(n/2)]*dx)/sum(probPsi*dx)
print('Probability for transmission, T = %f' % T)
print('Probability for reflection, R = %f' % R)
print('Sum = %f' % (T + R))

Можно также проанализировать, как трансмиссия зависит от различных свойств барьера. Например, вычисление вероятностей передачи и отражения для 50 различных высот барьера, $V_0$, от 0 до $3E/2$ с фиксированной шириной $L/20$.

In [ ]:
N = 50    # Number of different barriers
T = [0]*N # Define vector that holds the values for R and T
R = [0]*N
V0 = []

for q in range(N): # loop for each barrier property q
    
    # Setting the potential
    V = np.array([0]*n)
    length = L/50
    V0.append(3*E*q/(2*N))
    V[int(n/2-n*length/(2*L)):int(n/2+n*length/(2*L))] = V0[q]
    
    # Propagating wave function
    PsiI=C*np.exp(-(x - xs)**2/(2*sigma**2))*np.sin(k0*x - w*(-dt/2)) #\Psi_I(x,-dt/2)
    PsiR=C*np.exp(-(x - xs)**2/(2*sigma**2))*np.cos(k0*x - w*0)       #\Psi_R(x,0)
    PsiI[1] = PsiI[n-1] = PsiR[0] = PsiR[n-1] = 0
    
    for j in range(int((tmax)/dt)):
        PsiI, PsiR = timestep(PsiI, PsiR, V, dx, dt)
    
    probPsi = PsiI**2 + PsiR**2 #|\Psi|^2
    
    
    # Calculate transmission and reflection coefficients
    T[q] = sum(probPsi[int(n/2):int(n-1)]*dx)/sum(probPsi*dx)
    R[q] = sum(probPsi[0:int(n/2)]*dx)/sum(probPsi*dx)
    
# Plot the coefficients
plt.plot(np.array(V0)/E, T, label="$T$")
plt.plot(np.array(V0)/E, R, label="$R$")
plt.legend(loc=5)
plt.xlabel('$V_0/E$')
plt.axis([0, 1.5, -0.1, 1.1]);

Классически частица имела бы коэффициент пропускания, равный 1, когда $E>V_0$ и 0, когда $E<V_0$. Это не относится к квантовой механике, как видно из приведенного выше графика. Квантово-механически вероятность передачи равна 1, когда вообще нет барьеров. По мере увеличения прочности барьера коэффициент перехода стремится к 0. Обратите внимание, однако, что он никогда не достигает 0 и что существует конечная вероятность трансмиссии, даже если $E < V_0$. Это противоречивый квантово-механический эффект, известный как туннелирование. Имеет место синусоидальная тенденция коэффициентов, которая также может быть показана аналитически. Для некоторых соотношений длины волны к ширине/силе барьера барьер кажется "невидимым" для волны.

Также интересно рассчитать вероятности передачи и отражения для 50 различных ширин барьера от 0 до $L/20$ с фиксированной прочностью $V_0=9E/10$.

In [ ]:
N = 50    # Number of different barriers
T = [0]*N # Define vector that holds the values for R and T
R = [0]*N
length = []

for q in range(0,N): # loop for each barrier property q
    
    # Setting the potential
    V = np.array([0]*n)
    V0 = 9*E/10
    length.append(q*L/(20*N))
    V[int(n/2-n*length[q]/(2*L)):int(n/2+n*length[q]/(2*L))] = V0
    
    # Propagating wave function
    PsiI = C*np.exp(-(x - xs)**2/(2*sigma**2))*np.sin(k0*x - w*(-dt/2)) #\Psi_I(x,-dt/2)
    PsiR = C*np.exp(-(x - xs)**2/(2*sigma**2))*np.cos(k0*x - w*0) #\Psi_I(x,0)
    PsiI[1] = PsiI[n-1] = PsiR[0] = PsiR[n-1] = 0
    
    for j in range(int((tmax)/dt)):
        PsiI, PsiR = timestep(PsiI, PsiR, V, dx, dt)
    
    probPsi = PsiI**2 + PsiR**2 #|\Psi|^2
        
    # Calculate transmission and reflection coefficients
    T[q] = sum(probPsi[int(n/2):int(n-1)]*dx)/sum(probPsi*dx)
    R[q] = sum(probPsi[0:int(n/2)]*dx)/sum(probPsi*dx)
    
# Plot
plt.plot(np.array(length)/L, T, label="$T$")
plt.plot(np.array(length)/L, R, label="$R$")
plt.legend()
plt.xlabel('Length/L');

Классически переход всегда будет составлять $100\%$, так как $E>V_0$. Но в квантово-механическом случае это не так. Даже если $E>V_0$, всегда существует конечная вероятность отражения. Еще одно интересное явление, которое следует заметить, заключается в том, что большие волновые пакеты быстро распространяются (не только из-за дисперсии) через барьер или делятся на более мелкие. Это происходит потому, что части волнового пакета "застревают" в самом барьере в результате отражений.

### Периодические граничные условия

В предыдущих случаях волновая функция распространялась только через барьер посередине. На концах коробки волновая функция была установлена на $0$, что означает, что потенциал за пределами коробки бесконечен. Это приводит к отражению волнового пакета обратно. Чтобы проиллюстрировать это, мы проведем анализ, используя функции, определенные ниже.

In [ ]:
from matplotlib import animation
from IPython.display import HTML

# Set new figure parameters
newparams = {'axes.labelsize': 11, 'axes.linewidth': 0.5, 'savefig.dpi': 300, 
             'lines.linewidth': 1.0, 'figure.figsize': (2.5, 1.5),
             'ytick.labelsize': 5, 'xtick.labelsize': 5,}
plt.rcParams.update(newparams);

# First set up the figure, the axis, and the plot element we want to animate
fig = plt.figure()
ax = plt.axes(xlim=(0, 1), ylim=(0, 1))
line, = ax.plot([], [], lw=1)

# Initialize wavefunction and set boundary conditions
PsiI = C*np.exp(-(x - xs)**2/(2*sigma**2))*np.sin(k0*x - w*(-dt/2)) #\Psi_I(x,-dt/2)
PsiR = C*np.exp(-(x - xs)**2/(2*sigma**2))*np.cos(k0*x - w*0) #\Psi_I(x,0)
V = [0]*n

probPsi = [PsiR**2 + PsiI**2]

# Number of time steps
tmax = 2*L/(2*vg)

# Propagate in time
for j in range(int((tmax)/dt)):
    PsiI, PsiR = timestep(PsiI, PsiR, V, dx, dt)
    
    probPsi.append(PsiR**2 + PsiI**2)

# Reduce the number of points to animate
probPsi = probPsi[0:-1:100]

# Initialization function: plot the background of each frame
def init():
    line.set_data([], [])
    return line,

# Animation function which updates figure data.  This is called sequentially
def animate(i):
    line.set_data(x/L, probPsi[i])
    return line,

# Call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=len(probPsi), interval=30, blit=True)

# Display the animation
plt.close(anim._fig)
HTML(anim.to_html5_video())

Сейчас мы реализуем функцию ``timestep`` для использования вместо этого периодических граничных условий, т. е. $\Psi(x=0)=\Psi(x=L)$.

In [ ]:
def timestep_periodic(PsiI, PsiR, V, dx, dt):
    """ This function takes the current PsiI and PsiR and calculates the next value
    according to the theory section. 
    
    Input:
        PsiI    # Imaginary part of wavefunction at current timestep
        PsiR    # Real part of wavefunction at current timestep
        V       # Array with value of potential in the domain
        dx      # Grid size
        dt      # Size of time step
    
    Output:
        PsiI    # Imaginary part of wavefunction at next timestep
        PsiR    # Real part of wavefunction at next timestep
    """
    
    PsiI[1:n-1] = PsiI[1:n-1] - dt*(
        PsiR[1:n-1]*V[1:n-1]/hbar - hbar*(PsiR[2:n] - 2*PsiR[1:n-1] + PsiR[0:n-2])/(2*m*dx**2))
    PsiR[1:n-1] = PsiR[1:n-1] + dt*(
        PsiI[1:n-1]*V[1:n-1]/hbar - hbar*(PsiI[2:n] - 2*PsiI[1:n-1] + PsiI[0:n-2])/(2*m*dx**2))
    
    PsiI[n-1] = PsiI[n-1] - dt*(
        PsiR[n-1]*V[n-1]/hbar - hbar*(PsiR[1] - 2*PsiR[n-1] + PsiR[n-2])/(2*m*dx**2))
    PsiR[n-1] = PsiR[n-1] + dt*(
        PsiI[n-1]*V[n-1]/hbar - hbar*(PsiI[1] - 2*PsiI[n-1] + PsiI[n-2])/(2*m*dx**2))
    
    PsiI[0] = PsiI[n-1]
    PsiR[0] = PsiR[n-1]
    
    return PsiI, PsiR

In [ ]:
# Initialize wavefunction and set boundary conditions
PsiI = C*np.exp(-(x - xs)**2/(2*sigma**2))*np.sin(k0*x - w*(-dt/2)) #\Psi_I(x,-dt/2)
PsiR = C*np.exp(-(x - xs)**2/(2*sigma**2))*np.cos(k0*x - w*0) #\Psi_I(x,0)
V = [0]*n

probPsi = [PsiR**2 + PsiI**2]

# Number of time steps
tmax = 2*L/(2*vg)

# Propagate in time
for j in range(int((tmax)/dt)):
    PsiI, PsiR = timestep_periodic(PsiI, PsiR, V, dx, dt)
    
    probPsi.append(PsiR**2 + PsiI**2)

# Reduce the number of points to animate
probPsi = probPsi[0:-1:100]


# Call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=len(probPsi), interval=30, blit=True)

# Display the animation
plt.close(anim._fig)
HTML(anim.to_html5_video())

Мы видим, что волновой пакет успешно проходит через барьер при $x = L$ и появляется снова при $x = 0$, как и должно быть при использовании периодических граничных условий.

### Туннелирование

Туннелирование - это причина, по которой альфа-частицы способны вырваться из ядра атома, несмотря на недостаток энергии. Согласно классической механике, альфа-частица никак не может преодолеть потенциальный барьер, существующий в ядре, из-за электростатического притяжения. Однако, как мы увидим в анимации ниже, существует конечная вероятность того, что волновой пакет проникнет через барьер.

In [ ]:
# Initialize wavefunction and set boundary conditions
PsiI = C*np.exp(-(x - xs)**2/(2*sigma**2))*np.sin(k0*x - w*(-dt/2)) #\Psi_I(x,-dt/2)
PsiR = C*np.exp(-(x - xs)**2/(2*sigma**2))*np.cos(k0*x - w*0) #\Psi_I(x,0)
V = zeros(n)
V[int(n/2)::] = 1.5*E*np.exp(-np.linspace(0, 20, int(n/2)))
probPsi = [PsiR**2 + PsiI**2]


# Plot potential
plt.plot(x/L ,V/E, 'r', label='$V(x)/E$', lw=0.5)
plt.legend(prop={'size':8})
plt.xlabel('$x/L$')

# Number of time steps
tmax = L/(2*vg)

# Propagate in time
for j in range(int((tmax)/dt)):
    PsiI, PsiR = timestep(PsiI, PsiR, V, dx, dt)
    
    probPsi.append(PsiR**2 + PsiI**2)

# Reduce the number of points to animate
probPsi = probPsi[0:-1:100]


# Initialization function: plot the background of each frame
def init():
    line.set_data([], [])
    return line,

# Animation function which updates figure data.  This is called sequentially
def animate(i):
    line.set_data(x/L, probPsi[i])
    return line,

# Call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=len(probPsi), interval=30, blit=True)

# Display the animation
plt.close(anim._fig)
HTML(anim.to_html5_video())

Как мы видим, существует конечная вероятность перехода! Однако это очень грубая модель, поскольку кулоновский потенциал намного выше и имеет форму $1/x$, а не экспоненты. Однако, принимая во внимание, что альфа-частица чрезвычайно часто ударяется о барьер, увеличивая вероятность передачи, этот пример показывает, что вероятность излучения альфа-частицы значительна.

### References

This script is a part of a numerical exercise in the subject FY2045 - Quantum
Mechanics I at NTNU - Norwegian University of Science and Technology. The exercise
was prepared by Tor Nordam

[1] Nordam, T.: Numerical exercise, FY2045 Quantum Mechanics, http://amokk.phys.ntnu.no/files/FY2045_2015/exercise15/numerics.pdf, 27.09.2015, [Downloaded: 01.10.2015]

[2] Hemmer, P.C.: Kvantemekanikk, tapir akademisk forlag, 5. utg. 2005, Trondheim

[3] Roundy, D.: http://physics.oregonstate.edu/~roundyd/COURSES/ph365x/366.html, [Downloaded: 23.10.2015]

[4] Øverbø, I.: Lecture notes 2,3 and 7, http://amokk.phys.ntnu.no/FY2045/documents/, [Downloaded: 09.2015]